In [1]:
with open('./inputs/day11.txt', 'r') as f:
    stones = list(map(int, f.read().split()))

orig_stones = stones.copy()

stone_map = {}
def blink_stone(stone):
    if stone_map.get(stone) is None:
        if stone == 0:
            stone_map[stone] = [1]
        elif len(str(stone)) % 2 == 0:
            stone_map[stone] = [int(str(stone)[:len(str(stone)) // 2]), int(str(stone)[len(str(stone)) // 2:])]
        else:
            stone_map[stone] = [2024 * stone]

    return stone_map[stone]

def blink_stones(input_stones):
    output_stones = []
    for stone in input_stones:
        output_stones += blink_stone(stone)

    return output_stones

for i in range(25):
    stones = blink_stones(stones)

print('Answer to Day 11, Part 1:', len(stones))

stone_time_map = {}
def blink_length(stone, num_blinks):
    if num_blinks == 0:
        return 1
    elif stone_time_map.get((stone, num_blinks)) is None:
        ans = sum(blink_length(s, num_blinks-1) for s in blink_stone(stone))
        stone_time_map[(stone, num_blinks)] = ans
        return ans
    else:
        return stone_time_map[(stone, num_blinks)]

print('Answer to Day 11, Part 2:', sum(blink_length(s, num_blinks=75) for s in orig_stones))

Answer to Day 11, Part 1: 203228
Answer to Day 11, Part 2: 240884656550923


In [2]:
import networkx as nx

with open('./inputs/day12.txt', 'r') as f:
    m = f.read().split()

g = nx.Graph()
for i in range(len(m)):
    for j in range(len(m[i])):
        g.add_node((i, j), region=m[i][j])
        if i > 0 and m[i][j] == m[i-1][j]:
            g.add_edge((i, j), (i-1, j))
        if j > 0 and m[i][j] == m[i][j-1]:
            g.add_edge((i, j), (i, j-1))

print('Answer to Day 12, Part 1:', sum((2 * len(c) - len(g.subgraph(c).edges)) * 2 * len(c) for c in nx.connected_components(g)))

def get_num_sides(graph):
    border_edges = []
    for node in graph:
        all_possible_neighbors = set((
            (node[0]+1, node[1]),
            (node[0]-1, node[1]),
            (node[0], node[1]+1),
            (node[0], node[1]-1)
        ))
        for other_node in all_possible_neighbors - set(graph.neighbors(node)):
            border_edges.append((node, other_node))
    
    num_sides = len(border_edges)
    for i in range(len(border_edges)):
        edge = border_edges[i]
        if edge[0][0] == edge[1][0]:
            adjacent_edges = set((
                ((edge[0][0] + 1, edge[0][1]), (edge[1][0] + 1, edge[1][1])),
                ((edge[0][0] - 1, edge[0][1]), (edge[1][0] - 1, edge[1][1])),
            ))
        elif edge[0][1] == edge[1][1]:
            adjacent_edges = set((
                ((edge[0][0], edge[0][1] + 1), (edge[1][0], edge[1][1] + 1)),
                ((edge[0][0], edge[0][1] - 1), (edge[1][0], edge[1][1] - 1)),
            ))

        num_sides -= sum(e in border_edges[i+1:] for e in adjacent_edges)

    return num_sides

print('Answer to Day 12, Part 2:', sum(get_num_sides(g.subgraph(c)) * len(c) for c in nx.connected_components(g)))

Answer to Day 12, Part 1: 1352976
Answer to Day 12, Part 2: 808796


In [3]:
import re
from ortools.linear_solver import pywraplp
from ortools.sat.python import cp_model

problems = []
with open('./inputs/day13.txt', 'r') as f:
    for p in f.read().split('\n\n'):
        lines = p.split('\n')
        problems.append({
            'a': (
                int(re.search(r'(?<=X\+)[0-9]*', lines[0]).group(0)),
                int(re.search(r'(?<=Y\+)[0-9]*', lines[0]).group(0))
            ),
            'b': (
                int(re.search(r'(?<=X\+)[0-9]*', lines[1]).group(0)),
                int(re.search(r'(?<=Y\+)[0-9]*', lines[1]).group(0))
            ),
            'target': (
                int(re.search(r'(?<=X=)[0-9]*', lines[2]).group(0)),
                int(re.search(r'(?<=Y=)[0-9]*', lines[2]).group(0))
            )
        })

part1_ans = 0
for p in problems:
    solver = pywraplp.Solver.CreateSolver('SAT')
    a = solver.IntVar(0, 100, 'a')
    b = solver.IntVar(0, 100, 'b')

    solver.Add(a * p['a'][0] + b * p['b'][0] == p['target'][0])
    solver.Add(a * p['a'][1] + b * p['b'][1] == p['target'][1])
    solver.Minimize(3 * a + b)
    status = solver.Solve()
    if status == pywraplp.Solver.OPTIMAL:
        part1_ans += 3 * round(a.solution_value()) + round(b.solution_value())

print('Answer to Day 13, Part 1:', part1_ans)

part2_ans = 0
for p in problems:
    model = cp_model.CpModel()

    num_vals = 3
    a = model.new_int_var(0, 10000000000000, "a")
    b = model.new_int_var(0, 10000000000000, "b")

    model.add(a * p['a'][0] + b * p['b'][0] == 10000000000000 + p['target'][0])
    model.add(a * p['a'][1] + b * p['b'][1] == 10000000000000 + p['target'][1])
    model.minimize(3 * a + b)

    solver = cp_model.CpSolver()
    status = solver.solve(model)
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        part2_ans += 3 * solver.value(a) + solver.value(b)

print('Answer to Day 13, Part 2:', part2_ans)

Answer to Day 13, Part 1: 31623
Answer to Day 13, Part 2: 93209116744825
